In [16]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC

url = 'https://raw.githubusercontent.com/nparis87/MachineLearning/main/energy_etf.csv'

precio = pd.read_csv(url, sep=";", index_col=0, decimal=',')
precio.head()

,Cierre
Fecha,
11/20/2017,"$67,88"
11/27/2017,"$69,68"
12/4/2017,"$69,28"
12/11/2017,"$68,72"
12/18/2017,"$71,97"


In [17]:
precio.Cierre = precio.Cierre.str.replace('$', '').str.replace(',', '.').astype(float)

<ipython-input-17-fbc510357ecc>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  precio.Cierre = precio.Cierre.str.replace('$', '').str.replace(',', '.').astype(float)


## Convertir el índice en fecha

In [18]:
precio.index = pd.to_datetime(precio.index)

## Obtener retornos

In [19]:
retornos = precio.pct_change().dropna()

## Clasificar los resultados como una de tres clases: Retorno alcista +1, Retorno plano 0, Retorno bajista -1

In [20]:
def clasificar(x):
    if x > umbral:
        y = 1
    elif (x < umbral) and (x > -umbral):
        y = 0
    else:
        y = -1
    
    return y
    

In [33]:
umbral = 2

In [22]:
retornos['clase'] = retornos.Cierre.apply(clasificar)

In [29]:
retornos.head(10)
mat_retornos = retornos.Cierre.values

## Construimos el set de datos

In [60]:
per = 15 # ventana de tiempo - Columnas
muestras = 60

variables = np.zeros((1,per))
l = len(mat_retornos)

In [61]:
for x in range(muestras):
    variables = np.concatenate((variables, mat_retornos[l-per-x: l-x].reshape((1,per))))

In [64]:
variables.shape

(61, 15)

In [65]:
variables = variables[1:]

X = variables[:,:-1]
Y = variables[:,-1]*100

## Una manera interesante de modificar valores de matrices

In [66]:
neutral = [not x for x in ((Y>umbral)|(Y<-umbral))]

Y[Y>umbral] = 1
Y[Y<-umbral] = -1
Y[neutral] = 0

In [67]:
Y

array([ 0.,  0.,  1.,  1.,  1.,  0.,  1.,  1., -1., -1.,  0., -1.,  1.,
        0.,  1., -1.,  1.,  1., -1., -1.,  0., -1., -1.,  0.,  0.,  1.,
        0., -1.,  1.,  0., -1.,  0.,  1., -1.,  1., -1.,  1.,  1.,  0.,
       -1.,  1.,  1.,  1., -1.,  1.,  1.,  0.,  0., -1.,  1.,  0.,  0.,
       -1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.])

In [68]:
i_entrenamiento = int(len(variables)*0.7)
x_train = X[:i_entrenamiento, :]
x_test =  X[i_entrenamiento:, :]
y_train = Y[:i_entrenamiento]
y_test =  Y[i_entrenamiento:]

In [69]:
x_test.shape

(18, 14)

In [70]:
model = SVC().fit(x_train, y_train)

In [73]:
pred = model.predict(x_test)

In [75]:
(y_test == pred).sum()

7

In [76]:
help(model)

Help on SVC in module sklearn.svm._classes object:

class SVC(sklearn.svm._base.BaseSVC)
 |  SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)
 |  
 |  C-Support Vector Classification.
 |  
 |  The implementation is based on libsvm. The fit time scales at least
 |  quadratically with the number of samples and may be impractical
 |  beyond tens of thousands of samples. For large datasets
 |  consider using :class:`~sklearn.svm.LinearSVC` or
 |  :class:`~sklearn.linear_model.SGDClassifier` instead, possibly after a
 |  :class:`~sklearn.kernel_approximation.Nystroem` transformer.
 |  
 |  The multiclass support is handled according to a one-vs-one scheme.
 |  
 |  For details on the precise mathematical formulation of the provided
 |  kernel functions and how `gamma`, `coef0` and `degree` affect each
